In [2]:
import pandas as pd
import time
from dotenv import load_dotenv
load_dotenv("./.env.local")
import os
BEARER_TOKEN = os.getenv("BEARER_TOKEN")
import tweepy
client = tweepy.Client(BEARER_TOKEN)
from pathlib import Path

In [9]:
# Preprocessing the datasets and save them in csv files under csvdataframes folder
Path.mkdir(Path("csvdataframes"), exist_ok=True)
dataset_path = Path("dataset")

for idx, day in enumerate(dataset_path.iterdir()):
    full_dataset = pd.read_csv(day, compression='gzip')
    df_en = full_dataset[full_dataset['language']=='en']
    df_en_filteted = df_en[["userid", "tweetid", "text", "hashtags"]]
    df_no_duplicate = df_en_filteted.drop_duplicates(subset='text', keep='first')
    df_sampled = df_no_duplicate.sample(15000)
    df_sampled.to_csv(Path(f"csvdataframes/day_{idx}.csv"))

/tmp/ipykernel_4679/1859574095.py:3: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  full_dataset = pd.read_csv(day, compression='gzip')
/tmp/ipykernel_4679/1859574095.py:3: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  full_dataset = pd.read_csv(day, compression='gzip')


In [11]:
# Method to call on each csv day files
def create_edgelist(df_sample):  
    edgelist_retweet = open(f"edgelists/edgelist_retweet_{time.time()}.txt", "a")
    edgelist_retweet.write("tweet_id rt_user_id\n")

    edgelist_like = open(f"edgelists/edgelist_like_{time.time()}.txt", "a")
    edgelist_like.write("tweet_id like_user_id\n")

    for index, row in df_sample.iterrows():
        tweetid = row['tweetid']
        tweet = client.get_tweet(id=tweetid, expansions='referenced_tweets.id')
        og_tweet_id = tweet.includes['tweets'][0].id

        users_retweet = client.get_retweeters(id=og_tweet_id)
        for user in users_retweet.data:
            edgelist_retweet.write(f"{og_tweet_id} {user.id}\n")
            continue

        next_token = users_retweet.meta.get('next_token')

        n = 0
        while(next_token and n > 2):
            users_retweet = client.get_retweeters(id=og_tweet_id, next_token=next_token)
            for user in users_retweet.data:
                edgelist_retweet.write(f"{og_tweet_id} {user.id}\n")
                continue
            next_token = users_retweet.meta.get('next_token')

        users_like = client.get_liking_users(id=og_tweet_id)
        for user in users_like.data:
            edgelist_like.write(f"{og_tweet_id} {user.id}\n")
            continue
        n = n + 1 
        next_token = users_like.meta.get('next_token')

        n = 0
        while(next_token and n > 2):
            users_like = client.get_retweeters(id=og_tweet_id, next_token=next_token)
            for user in users_like.data:
                edgelist_like.write(f"{og_tweet_id} {user.id}\n")
                continue
            n = n + 1 
            next_token = users_like.meta.get('next_token')

    edgelist_retweet.close()
    edgelist_like.close()

In [ ]:
# Main loop
Path.mkdir(Path("edgelists"), exist_ok=True)
for file in Path("csv_dataframes").iterdir():
    df = pd.read_csv(file)
    create_edgelist(df)